# Import

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import datetime
import matplotlib.pyplot as plt
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import ndcg_score
from sklearn.ensemble import RandomForestRegressor
from tqdm.auto import tqdm
import joblib  # 直接导入 joblib



# Model init

In [2]:
file_path = 'C:/Users/98398/Desktop/P5/DM-AS2/Test_data/feature_engineered_training_chunk_0.csv'

#file_path = 'C:/Users/98398/Desktop/P5/DM-AS2/Test_data/cleaned_training_set_VU_DM.csv'
#file_path = 'C:/Users/98398/Desktop/P5/DM-AS2/Test_data/trai ning_set_VU_DM.csv'
df = pd.read_csv(file_path)

df_100 = pd.read_csv(file_path, nrows=100)
#print(df_100.head())
# 增加一个计算得分的列
def assign_scores(row):
    if row['booking_bool'] == 1:
        return 5
    elif row['click_bool'] == 1:
        return 1
    else:
        return 0

df['score'] = df.apply(assign_scores, axis=1)

   srch_id            date_time  site_id  visitor_location_country_id  \
0        1  2013-04-04 08:32:15       12                          187   
1        1  2013-04-04 08:32:15       12                          187   
2        1  2013-04-04 08:32:15       12                          187   
3        1  2013-04-04 08:32:15       12                          187   
4        1  2013-04-04 08:32:15       12                          187   

   visitor_hist_starrating  visitor_hist_adr_usd  prop_country_id  prop_id  \
0                      0.0                   0.0              219      893   
1                      0.0                   0.0              219    10404   
2                      0.0                   0.0              219    21315   
3                      0.0                   0.0              219    27348   
4                      0.0                   0.0              219    29604   

   prop_starrating  prop_review_score  ...  \
0                3                3.5  ...   


In [3]:
columns = df.columns

# 使用列表推导式筛选出不需要删除的列
features = [
    col for col in columns if col not in ['date_time', 'position', 'click_bool', 'booking_bool', 'score']
    and 'gross_bookings_usd' not in col and 'position' not in col
]


print(len(features))
print(len(df.columns))

257
281


In [ ]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['srch_id'])

# 填充训练数据中的 NaN 值为 0
train_df[features] = train_df[features].fillna(0)
train_df['score'] = train_df['score'].fillna(0)

# 用同样的方法处理测试数据集的 NaN 值
test_df[features] = test_df[features].fillna(0)
print(1)

In [10]:
# 创建一个包装器来报告进度的函数
def progress_display(n_total):
    pbar = tqdm(total=n_total)
    def update(*args):
        pbar.update()
    return update, pbar

# 设置回调函数来监控树的构建进度
n_trees = 100
update_callback, pbar = progress_display(n_trees)
with joblib.parallel_backend('threading', n_jobs=-1):
    rf = RandomForestRegressor(n_estimators=n_trees, max_depth=10, random_state=42, verbose=0, n_jobs=-1)
    with joblib.parallel_backend('threading', n_jobs=-1):
        rf.fit(train_df[features], train_df['score'])
pbar.close()


test_pred = rf.predict(test_df[features])
test_df['predictions'] = test_pred
print(2)

# 首先确保数据按照 srch_id 和 predictions 降序排序
test_df.sort_values(['srch_id', 'predictions'], ascending=[True, False], inplace=True)

# 分组并计算每个搜索会话的 NDCG
grouped = test_df.groupby('srch_id')
ndcg_scores = []
print(3)
for name, group in grouped:
    group = group.sort_values('predictions', ascending=False)
    true_relevance = group['score'].values
    scores_pred = group['predictions'].values
    # 只计算有正样本的会话
    if len(np.unique(true_relevance)) > 1:
        ndcg_scores.append(ndcg_score([true_relevance], [scores_pred], k=5))

average_ndcg = np.mean(ndcg_scores)
print(f"Average NDCG Score: {average_ndcg}")


1


  0%|          | 0/100 [00:00<?, ?it/s]

2
3
Average NDCG Score: 0.6825715630838168
